# Учебно-тренировочная нейронная сеть.

ЗАДАЧА - научиться по фотографии лица человека предсказывать его пол, расу и возраст.

[Датасет](https://susanqq.github.io/UTKFace/), который мы будем использовать состоит из 20000 фотографий лиц людей в возрасте от 0 до 116 лет! В качестве разметки имеется пол (male/female), раса (white/black/asian/indian/other) и возраст. Классификация с настолько сильной внутриклассовой изменьчивостью (от младенцев до пожилых людей!) -- сложная задача. 

Мы немного упростим себе задачу и воспользуемся версией датасета, где все картинки приведены к одному размеру и лица находятся в центре изображения.

Как решать такую задачу? **Transfer learning и finetuning**. Т.е. возьмем за основу сеть, обученную на большом количестве данных для другой задачи и дообучим ее для нашей задачи.



Мы возьмем за основу сеть, которая была обучена на датасете [VGGFace2](http://www.robots.ox.ac.uk/~vgg/data/vgg_face2/) для классификации лиц. Он содержит 3.3 миллиона изображений с 9000 разными персоналиями.


## Загрузка обученной модели

Сеть, которую мы будем дообучать мы возьмем из открытого источника: https://github.com/rcmalli/keras-vggface .

In [0]:
! pip install git+https://github.com/rcmalli/keras-vggface.git

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
# tf.enable_eager_execution()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Сохраним модель с помощью save_model. Она находится по адресу: https://drive.google.com/open?id=1oHJxVZCcVwp1dgcwDIZL4h97uInxOGWO . Загрузим модель:


In [0]:
! pip install gdown
import gdown

url = 'https://drive.google.com/uc?id=1oHJxVZCcVwp1dgcwDIZL4h97uInxOGWO'
output = 'resnet50face.h5'
gdown.download(url, output, quiet=False)

In [0]:
from tensorflow.keras.models import load_model
vggface_model = load_model("resnet50face.h5")

Модель загружена, а теперь посмотрим, что она из себя представляет. Это архитектура -- ResNet. На самом деле внутренности модели не очень важны для нас -- в дальнейшем мы заморозим бОльшую часть этой сети и будем работать только с последними слоями. Нам важно, что она обучена и мы имеем доступ к ее архитектуре и весам.

In [0]:
vggface_model.summary() # последний слой классифицирует на 8631 классов

Для проверки того, что помимо архитектуры мы правильно загрузили веса, проверим предсказания модели на следующем изображении: 

In [0]:
! wget https://img.joinfo.ua/i/2019/01/5c4ea940d2b08.jpg -O brad_pitt.jpg

После загрузки запустим полный пример из репозитория:

In [0]:
from keras_vggface import utils
from tensorflow.keras.preprocessing import image

img = image.load_img("brad_pitt.jpg", target_size=(224, 224)) # модель работает с картинками размера 224 на 224
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = utils.preprocess_input(x, version=2) # мы используем resnet50 -- поэтому version2. Эта функция нам еще пригодится!

preds = vggface_model.predict(x) # модель -- известная нам keras model, вызываем predict
plt.title(f"Predicted: {utils.decode_predictions(preds)[0][0]}") # используем  decode_predictions из keras_vggface
plt.imshow(img)

## Загрузка данных
Мы убедились, что модель загружена правильно, узнали какой препроцессинг ей необходим (utils.preprocess_input(x, version=2)) и теперь можем перейти к загрузке наших данных.

In [0]:
from pathlib import Path
from collections import Counter

In [0]:
url = 'https://drive.google.com/uc?id=0BxYys69jI14kYVM3aVhKS1VhRUk'
output = '/tmp/UTKFace.tar.gz'
gdown.download(url, output, quiet=False)
! tar -xzf /tmp/UTKFace.tar.gz -C /tmp/
url = 'https://drive.google.com/uc?id=1mux7xiP4NP6AUUFvUW42RgSdUEQ4k5e2'
output = 'train_val_split_utk.csv'
gdown.download(url, output, quiet=False)

In [0]:
data_folder = Path("/tmp/UTKFace/")
filenames = list(map(lambda x: x.name, data_folder.glob('*.jpg')))
print(len(filenames))
print(filenames[:3]) # имя файла содержит возраст, индекс пола и расы, перечисленные через нижнее подчеркивание

In [0]:
# формируем датафрейм с путями и нужными классами
np.random.seed(10)
np.random.shuffle(filenames)
gender_mapping = {0: 'Male', 1: 'Female'}
race_mapping = dict(list(enumerate(('White', 'Black', 'Asian', 'Indian', 'Others'))))
age_labels, gender_labels, race_labels, correct_filenames = [], [], [], []

for filename in filenames:
    if len(filename.split('_')) != 4:
        print(f"Bad filename {filename}")
        continue

    age, gender, race, _ = filename.split('_')
    correct_filenames.append(filename)
    age_labels.append(age)
    gender_labels.append(gender)
    race_labels.append(race)
    
age_labels = np.array(age_labels, dtype=np.float32)
max_age = age_labels.max()
data = {"img_name": correct_filenames, 
        "age": age_labels / max_age, # нормализуем возраст от 0 до 1
        "race": race_labels, 
        "gender": gender_labels}
df = pd.DataFrame(data)
df.head()

In [0]:
df_split = pd.read_csv("train_val_split_utk.csv", index_col=0).set_index("img_name")
df_split.head()
df = df.set_index("img_name").join(df_split).reset_index()
df_train = df[df["is_train"] == 1]
df_val = df[df["is_train"] != 1]
print(len(df_val), len(df_train))

In [0]:
df_split.head()

In [0]:
df_train.head()

In [0]:
df_val.head()

In [0]:
def decode_labels(race_id, gender_id, age):
    return race_mapping[int(race_id)], gender_mapping[int(gender_id)], int(age*max_age)
def show_face(image, race_id, gender_id, age):
    plt.imshow(image)
    race, gender, age = decode_labels(race_id, gender_id, age)
    plt.title(f"Gender: {gender}, Race: {race}, Age: {age}")

Убедимся, что метки классов корректны:

In [0]:
row = df.iloc[np.random.randint(len(df))]
img = plt.imread(str(data_folder / row["img_name"]))
show_face(img, row["race"], row["gender"], row["age"])

Посмотрим на их распределение:

In [0]:
races_verbose = [race_mapping[int(race)] for race in df["race"]]
genders_verbose = [gender_mapping[int(gender)] for gender in df["gender"]]

print(Counter(races_verbose))
print(Counter(genders_verbose))
_ = plt.hist(df["age"]*max_age)
_ = plt.xlabel("Age")

## Создание генератора данных для модели предсказания пола.
Предподготовка закончена. Реализуем генератор данных необходимый для последущего обучения модели. `tensorflow.keras.preprocessing.image.ImageDataGenerator`

Как мы знаем, перед применением обученных сетей нужно знать какой препроцессинг они проводят с данными. Т.к. эта модель не часть Keras, мы должны ответить на этот вопрос сами с помощью исходников автора этой сети. К счастью, он приводит пример использования на главной странице репозитория. Видно, что он использует функцию preprocess_input из utils. Значит ей мы и должны воспользоваться!

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def preprocess_input_facenet(image):
    """
    image -- тензор размера (1, H, W, 3)
    
    return: картинка, с примененным preprocess_input(..., version=2) из keras_vggface (см пример с Бредом Питом)
    """

    x = np.expand_dims(image, axis=0)
    x = utils.preprocess_input(x, version=2)

    assert x.shape == (1, 224, 224, 3), 'Wrong preprocessed image shape!'
    return x

# image_gen должен содержать ImageDataGenerator с правильной preprocessing_function

image_gen = ImageDataGenerator(preprocessing_function=preprocess_input_facenet)

Для генерации картинок раньше мы использовали `image_gen.flow_from_folder`. Но для этого данные должны быть расположены на диске с определенной структурой папок, что в данном случае не очень удобно. Поэтому мы должны воспользоваться более гибким `image_gen.flow_from_dataframe`, который позволяет генерировать данные с нужными классами используя датафрейм.

**Документация по [ссылке](https://keras.io/preprocessing/image/) (раздел flow_from_dataframe), создадим train_generator и val_generator для df_train и df_val соответственно.**
*  Генератор должен возвращать картинку и ее класс (male/female)
*  class_mode "binary", directory=str(data_folder)
*  batch_size, image_size указаны ниже
*  train_generator должен перемешивать данные, а val_generator -- не должен
*  После выполнения клетки ниже мы увидим текст: 

*Found 18946 validated image filenames belonging to 2 classes.
Found 4759 validated image filenames belonging to 2 classes.*




In [0]:
BATCH_SIZE = 128
IMAGE_SIZE = 224

# train_generator = image_gen.flow_from_dataframe( ... ), 
# val_generator = image_gen.flow_from_dataframe( ... )

train_generator = image_gen.flow_from_dataframe(df_train, 
                                                directory=str(data_folder), 
                                                x_col='img_name', 
                                                y_col='gender', 
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                class_mode='binary')

val_generator = image_gen.flow_from_dataframe(df_val, 
                                              directory=str(data_folder), 
                                              x_col='img_name', 
                                              y_col='gender',
                                              batch_size=BATCH_SIZE, 
                                              shuffle=False, 
                                              target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                              class_mode='binary')

In [0]:
sample_images, sample_labels = next(val_generator)
assert sample_images.shape == (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), "Неправильный размер батча"
assert sample_labels.shape == (BATCH_SIZE,), "Неправильный размер меток класса"
assert list(sorted(np.unique(sample_labels))) == [0., 1.], "Ожидаемые классы 0 и 1"
print("Simple tests passed")

Попробуем отобразить картинку и ее лейбл для проверки генерации:

In [0]:
plt.imshow(sample_images[100])
print(gender_mapping[int(sample_labels[100])])
# мы получим очень странное изображение и это нормально. 
# matplotlib ожидает картинку с интесивностями от 0 до 1 если она типа float и от 0 до 255 если int.

In [0]:
print(sample_images[100].max(), sample_images[100].min()) # можно увидеть что значения яркостей типа float и в т.ч. отрицательные

**Реализуем функцию `deprocess_image`, которая преобразует картинку используемую keras_vggface обратно к изображению, которое можно визуализировать с помощью `matplotlib`.**
*  нам понадобится https://github.com/rcmalli/keras-vggface/blob/master/keras_vggface/utils.py
*  необходимо проделать операции preprocess_input в обратном порядке
*  наш случай: version=2, format="channels_last"

In [0]:
def deprocess_image(vggface_image):
    """
    vggface_image -- (H, W, 3) картинка после препроцессинга. 
    содержит отрицательные значения и некорректно отображается matplotlib

    return: корректно отображаемая картинка типа np.uint8(!!). 

    ! работаем с копией картинки (image = np.copy(vggface_image)) !
    """
    
    image = np.copy(vggface_image) 

    image[..., 0] += 91.4953
    image[..., 1] += 103.8827
    image[..., 2] += 131.0912
    image = image[..., ::-1]

    return np.uint8(image)

# теперь картинка должна отображаться корректно
plt.imshow(deprocess_image(sample_images[5]))
print(gender_mapping[int(sample_labels[5])])

### Обучение модели предсказания пола

Мы обучим модель предсказания пола с точностью более 90% на валидационном сете. 

*   Возьмем за основу 'base_model' (определена ниже)
*   Мы сами решаем сколько слоев замораживать и сколько полносвязанных слоев использовать. 
*   Используем чекпоинты, чтобы не потерять веса лучшей модели. Они должны быть сохранены с именем "model_gender/checkpoint_best.h5"
*   В model.fit(...) используем steps_per_epoch=25, для того чтобы проверка на валидации происходила чаще и мы могли более точно отслеживать прогресс. 

In [0]:
# для начала "отрежем" от vggface_model последний слой классификатора 
# теперь для картинки base_model предсказывает 2048-мерный вектор признаков.

base_model = tf.keras.Model([vggface_model.input], vggface_model.get_layer("flatten_1").output)
base_model.summary()

In [0]:
!pip install livelossplot
from livelossplot.tf_keras import PlotLossesCallback

In [0]:
from google.colab import drive
from pathlib import Path

# прикрепим гугл диск к виртуальной машине
drive.mount('/content/drive/')

In [0]:
# следующие преобразования 
path = Path("/content/drive/My Drive/model_gender")
path.mkdir(exist_ok=True, parents=True) 
cpt_filename = "checkpoint_best.h5"  
cpt_gender =str(path / cpt_filename)

In [0]:
# определяем чекпоинт 
checkpoint = tf.keras.callbacks.ModelCheckpoint(cpt_gender, 
                                                monitor='val_acc', 
                                                verbose=1, 
                                                save_best_only=True, 
                                                mode='max')

In [0]:
# Обучение модели предсказания пола

base_model.trainable = False # замораживаем всю базовую модель

model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [0]:
EPOCHS = 15
history = model.fit_generator(train_generator, steps_per_epoch=25, 
                              epochs=EPOCHS, 
                              validation_data=val_generator, 
                              callbacks=[PlotLossesCallback(), checkpoint])

In [0]:
# попробуем улучшить 
base_model.trainable = True
print("Количество слоев в базовой модели: ", len(base_model.layers))

In [0]:
fine_tune_at = 160
# все слои до -- заморозим
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [0]:
EPOCHS = 10
history = model.fit_generator(train_generator, steps_per_epoch=25, 
                              epochs=EPOCHS, 
                              validation_data=val_generator, 
                              callbacks=[PlotLossesCallback(), checkpoint])

In [0]:
model_gender = load_model(cpt_gender)

loss, acc = model_gender.evaluate(val_generator)
if acc < 0.9:
    print("Please, try harder!")
else:
    if acc >= 0.94:
        print("Well done!")
    else:
        print("Very good! Can you improve accuracy?")

In [0]:
#@title (вспомогательный код, выполните клетку)
def show_faces(images, real_race=None, real_gender=None, real_age=None, 
               predicted_race=None, predicted_gender=None, predicted_age=None):
    plt.figure(figsize=(10,10))
    labels = {"Gender": [predicted_gender, real_gender],
                  "Race": [predicted_race, real_race],
                  "Age": [predicted_age, real_age]}
    for i in range(16):
        plt.subplot(4,4, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(deprocess_image(images[i]))
        real_str = "Real:"
        pred_str = "Pred:"
        correct = True
        for name, (predicted, real) in labels.items():
            if predicted is None:
                continue
            if name == "Age":
                real_age = int(real[i]*int(max_age))
                predicted_age = int(predicted[i]*max_age)
                real_str += f"{real_age}"
                pred_str += f"{predicted_age}"
                if np.abs(predicted_age - real_age) > 6:
                    correct = False

            elif name == "Gender":
                real_gender = int(real[i])
                predicted_gender = int(predicted[i] > 0.5)
                real_str += f"{gender_mapping[real_gender]}, "
                pred_str += f"{gender_mapping[predicted_gender]}, "
                if real_gender != predicted_gender:
                    correct = False
            elif name == "Race":
                real_race = int(real[i])
                predicted_race = np.argmax(predicted[i])
                real_str += f"{race_mapping[real_race]}, "
                pred_str += f"{race_mapping[predicted_race]}, "
                if real_race != predicted_race:
                    correct = False
                
        title_obj = plt.title(f"{real_str}\n{pred_str}")
            
        plt.subplots_adjust(wspace=0.4)
        if not correct:
            plt.setp(title_obj, color='r')
        

In [0]:
sample_validation_images, sample_validation_labels = next(val_generator)
predicted = model_gender.predict(sample_validation_images)
show_faces(sample_validation_images, real_gender=sample_validation_labels, predicted_gender=predicted)

### Обучение модели предсказания расы

Мы обучим модель предсказания расы с точностью более 80% на валидационном сете. 

*   Используя код аналогичный тому, что выше, реализуем модель предсказания расы
*   Для начала определим генератор данных
    *  нужно указать class_mode="sparse" и изменить "y_col"
*   Используем чекпоинты, чтобы не потерять веса лучшей модели. Они должны быть сохранены с именем "model_race/checkpoint_best.h5"

In [0]:
# генераторы данных модели предсказания рас 
train_gen_race = image_gen.flow_from_dataframe(df_train, 
                                               directory=str(data_folder), 
                                               x_col='img_name', 
                                               y_col='race', 
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               class_mode='sparse')

val_gen_race = image_gen.flow_from_dataframe(df_val, 
                                             directory=str(data_folder), 
                                             x_col='img_name', 
                                             y_col='race',
                                             batch_size=BATCH_SIZE, 
                                             shuffle=False, 
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                             class_mode='sparse')

In [0]:
# vggface_model = load_model("resnet50face.h5")
base_model = tf.keras.Model([vggface_model.input], 
                            vggface_model.get_layer("flatten_1").output)

In [0]:
# преобразования 
path = Path("/content/drive/My Drive/model_race")
path.mkdir(exist_ok=True, parents=True)
cpt_filename = "checkpoint_best.h5"  
cpt_race =str(path / cpt_filename)

In [0]:
# определяем чекпоинт 
checkpoint_2 = tf.keras.callbacks.ModelCheckpoint(cpt_race, 
                                                  monitor='val_acc', 
                                                  verbose=1, 
                                                  save_best_only=True, 
                                                  mode='max')

In [0]:
# Обучение модели предсказания расы

base_model.trainable = False # замораживаем всю базовую модель

model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              loss="sparse_categorical_crossentropy", 
              metrics=['accuracy'])

model.summary()

In [0]:
EPOCHS = 15 
history = model.fit_generator(train_gen_race, steps_per_epoch=25, 
                              epochs=EPOCHS, 
                              validation_data=val_gen_race, 
                              callbacks=[PlotLossesCallback(), checkpoint_2])

In [0]:
# попробуем улучшить 
base_model.trainable = True
print("Количество слоев в базовой модели: ", len(base_model.layers))

In [0]:
fine_tune_at = 160
# все слои до -- заморозим
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
              loss="sparse_categorical_crossentropy", 
              metrics=['accuracy'])

model.summary()

In [0]:
EPOCHS = 10 
history = model.fit_generator(train_gen_race, steps_per_epoch=25, 
                              epochs=EPOCHS, 
                              validation_data=val_gen_race, 
                              callbacks=[PlotLossesCallback(), checkpoint_2])

In [0]:
model_race = load_model(cpt_race)
loss, acc = model_race.evaluate(val_gen_race)
if acc < 0.8:
    print("Please, try harder!")
else:
    if acc >= 0.85:
        print("Well done!")
    else:
        print("Very good! Can you improve accuracy?")

In [0]:
sample_validation_images, sample_validation_labels = next(val_gen_race)
predicted = model_race.predict(sample_validation_images)
show_faces(sample_validation_images, real_race=sample_validation_labels, predicted_race=predicted)

## Multitask learning

Мы могли бы обучить регрессор возраста по картинке, используя аналогичный уже написаному код. Но мы пойдем по другому пути. 

Если на каждый атрибут (возраст, раса, пол) создавать одну модель, то таких моделей получится 3. Очевидно, что для предсказания на новой картинке необходимо будет произвести инференс трех сетей. Это может быть ресурсо-затратно и сложнее в поддержке. 

Какой же есть еще подход? Он называется Multitask learning. Его идея в том, чтобы обучать одну модель для решения сразу нескольких задач! Гибкость нейронных сетей позволяет это сделать достаточно логичным образом. Необходимо вместо одной "головы" для предсказания сделать несколько. Таким образом в нашей задаче одна будет отвечать за классификацию пола (1 выход, вероятность), вторая -- за классификацию расы (5 выходов, вероятности) и третья -- за предсказания возраста (1 выход, число).

**Реализуем модель сети с тремя "головами".**

*    Нам нужно вспомнить что такое Functional API для определения моделей, потому что необходимо определить архитектуру с тремя выходами. С помощью Sequential API этого сделать нельзя.
*    За основу возьмем base_model (определена ниже), добавим к ее выходу 1-2 полносвязных слоя (количество нейронов определим самостоятельно)
*    К последнему слою добавим три паралельных выхода: два с одним нейроном и один с пятью
*    При определении выходных слоев надо указать Dense(..., name="race"/"age"/"gender" )
*    !Правильные активации! Возраст у нас нормирован (от 0 до 1). 
*    Порядок выходов модели: `outputs=[gender_output, race_output, age_output]`
*    Точность предсказания пола должна быть > 90%, расы > 80% а MAE для возраста < 0.09.

In [0]:
# vggface_model = load_model("resnet50face.h5")
base_model = tf.keras.Model([vggface_model.input], vggface_model.get_layer("flatten_1").output)
base_model.trainable = False

In [0]:
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers
from tensorflow import keras
from keras import regularizers

In [0]:
input_layer = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

# переменная model должна содержать модель для дальнейшего обучения 
base_l = base_model(input_layer)
x = layers.Dense(2048, activation='relu', 
                 kernel_regularizer=regularizers.l2(0.001), 
                 activity_regularizer=regularizers.l1(0.01), 
                 name='dense_1')(base_l)

gender_output = layers.Dense(1, activation = 'sigmoid', name = 'gender')(x)
race_output = layers.Dense(5, activation = 'softmax', name = 'race')(x)
age_output = layers.Dense(1, activation = 'relu', name = 'age')(x)

model = keras.Model(inputs=input_layer, 
                    outputs=[gender_output, race_output, age_output])

model.summary()
plot_model(model)

In [0]:
prediction = model(np.zeros((6, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32))
assert len(prediction) == 3, "Модель должна возвращать три тензора"
assert prediction[0].shape == (6, 1), f"Неправильный размер выхода gender: {prediction[0].shape}"
assert prediction[1].shape == (6, 5), f"Неправильный размер выхода race: {prediction[1].shape}"
assert prediction[2].shape == (6, 1), f"Неправильный размер выхода age: {prediction[2].shape}"
print("Shape tests passed")

Необходимые генераторы данных. Обратим внимание на y_col и class_mode.

In [0]:
train_generator = image_gen.flow_from_dataframe(
        dataframe=df_train,
        class_mode="other",
        x_col="img_name", y_col=["gender", "race", "age"], # нас интересуют все три столбца
        directory=str(data_folder),
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        shuffle=True)

val_generator = image_gen.flow_from_dataframe(
        dataframe=df_val,
        class_mode="other",
        x_col="img_name", 
        y_col=["gender", "race", "age"], 
        directory=str(data_folder),
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        shuffle=True)

def split_outputs(generator):
    """
    Вспомогательная функция, которая модернизирует генераторы картинок, чтобы их
    можно было использовать для Мultitask
    image_gen.flow_from_dataframe возвращает на каждой итерации батч:
    ((N, H, W, 3), (N, 3)) -- N картинок и N троек меток (для трех "задач")
    model.fit(..) ожидает генератор в формате:
    ((N, H, W, 3), [(N, 1), (N, 1), (N, 1)])

    Для такого превращения и нужна эта функция.

    """
    while True:
        data = next(generator)
        image = data[0]
        labels = np.split(data[1], 3, axis=1)
        yield image, labels

А теперь перейдем к обучению:

In [0]:
path = Path("/content/drive/My Drive/model_multitask")
path.mkdir(exist_ok=True, parents=True) 
cpt_filename = "checkpoint_best.h5"  
cpt_multitask =str(path / cpt_filename)

checkpoint_3 = tf.keras.callbacks.ModelCheckpoint(cpt_multitask, 
                                                  monitor='val_age_mean_absolute_error', 
                                                  verbose=1, 
                                                  save_best_only=True, 
                                                  mode='min')

In [0]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) 
# можно выбрать и другую скорость обучения 
### 
# !теперь loss -- это словарь, в котором к каждому выходу мы "прицепляем" свой лосс!
# аналогично с metrics
model.compile(optimizer=optimizer, 
              loss={'gender': 'binary_crossentropy', 
                    'race': 'sparse_categorical_crossentropy', 
                    'age': 'mse'},
              metrics={'gender': 'accuracy', 
                       'race': 'accuracy', 
                       'age': 'mae'})

model.fit_generator(split_outputs(train_generator), epochs=90, 
                    validation_data=split_outputs(val_generator), 
                    callbacks=[PlotLossesCallback(), checkpoint_3], 
                    steps_per_epoch=50, 
                    validation_steps= len(df_val) // BATCH_SIZE)

In [0]:
model_multitask = load_model(cpt_multitask)

results = model.evaluate(split_outputs(val_generator), steps=len(df_val)//BATCH_SIZE)
assert results[-3] > 0.90, f"Gender accuracy is too low. Please try to improve it {results[-3]}"
assert results[-2] > 0.80, f"Race accuracy is too low. Please try to improve it. {results[-2]}"
assert results[-1] < 0.09, f"Age MAE it too high: {results[-1]}"
print("Well done!")

In [0]:
sample_validation_images, sample_validation_labels = next(split_outputs(val_generator))
predicted = model_multitask.predict(sample_validation_images)
show_faces(sample_validation_images, 
           real_gender=sample_validation_labels[0], predicted_gender=predicted[0],
           real_race=sample_validation_labels[1], predicted_race=predicted[1],
           real_age=sample_validation_labels[2].flatten(), predicted_age=predicted[2].flatten(),
           )

Загрузим любое лицо и получим предсказание. Не забудем его обрезать соответствующим образом.

In [0]:
url = 'google_disk'
output = 'imagge.jpg'
gdown.download(url, output, quiet=False)

img = image.load_img("imagge.jpg", target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = utils.preprocess_input(x, version=2) 
predicted_labels = model.predict(x)
plt.imshow(img)
gender, race, age = int(predicted_labels[0][0] > 0.5), np.argmax(predicted_labels[1][0]), predicted_labels[2][0]
title_obj = f"Pred: {gender_mapping[gender]}, {race_mapping[race]}, {int(age[0]*max_age)}."
_ = plt.title(title_obj)